In [ ]:
!pip install datasets
!pip install nltk

In [ ]:
from datasets import load_dataset

dataset_train = load_dataset('imdb', split='train')
dataset_test = load_dataset('imdb', split='test')
dataset_unsupervised = load_dataset('imdb', split='unsupervised')

In [ ]:
set(dataset_train['label'])

In [ ]:
dataset_test['label'][12490:12010]

In [ ]:
dataset_train['label'].count(0), dataset_train['label'].count(1), dataset_test['label'].count(0), dataset_test['label'].count(1), 

Naive Bayes classifier

(2 points) Take a look at the data and create an adapted preprocessing function which at least:
- Lower case the text.

In [ ]:
def to_lower(in_list: list) -> str:
  return list(map(str.lower, in_list))

In [ ]:
lower_train = to_lower(dataset_train['text'])
len(dataset_train['text'])

- Remove punctuation (you can use from string import punctuation to ease your work).

In [ ]:
from string import punctuation
def remove_punct(in_list: list) -> str:
  return list(map(lambda input : input.translate(str.maketrans('', '', punctuation)), in_list))

(4 points) Implement and train a naive Bayes classifier on the training data. Either:
- We used a scikit-learn Pipeline with a CountVectorizer and MultinomialNB classifier.

In [ ]:
X_train = remove_punct(lower_train)
X_train[0]

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
vectorizer_train = CountVectorizer()
X = vectorizer_train.fit_transform(X_train)
y_train = dataset_train['label']

In [ ]:
X[:5].toarray()

In [ ]:
clf = MultinomialNB()
clf.fit(X, dataset_train['label'])
print(clf.predict(X[100]))

In [ ]:
#pipeline tests
lower_test = to_lower(dataset_test['text'])
no_punct_test =  remove_punct(lower_test)
X_test = vectorizer_train.transform(no_punct_test)

In [ ]:
print(clf.predict(X_test[20000:20050]))

In [ ]:
pipeline = make_pipeline(CountVectorizer(), MultinomialNB())
pipeline, pipeline.steps

In [ ]:
pipeline.fit(X_train, y_train)
pipeline.score(X_train, y_train)

In [ ]:
X_test = no_punct_test
y_test = dataset_train['label']

(1 point) Report the accuracy on both training and test set.

In [ ]:
pipeline.score(X_test, y_test)

(1 point) Why is accuracy a sufficient measure of evaluation here?

Accuracy is sufficient in our situation since we only have two classes.

[Bonus] What are the top 10 most important words (features) for each class? (bonus points

In [ ]:
ranks = []
for prob in pipeline[1].feature_log_prob_:
  ranks.append(prob.argsort()[-10:])
ranks

Remove stopwords (see NLTK stopwords corpus) and check 

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stops = set(stopwords.words('english'))
def remove_stop_words(corpus):
  no_stop_reviews = []
  for review in corpus:
    tokenized = word_tokenize(review)
    fix = []
    for word in tokenized:
      if word not in stops:
        fix.append(word)
    no_stop_reviews.append(' '.join(fix))
  return no_stop_reviews

X_train_no_stop = remove_stop_words(X_train)

(2 points) Add stemming or lemmatization to your pretreatment.

In [ ]:
no_stop_reviews = []
for review in X_train:
  tokenized = word_tokenize(review)
  fix = []
  for word in tokenized:
    if word not in stops:
      fix.append(word)
  no_stop_reviews.append(' '.join(fix))
no_stop_reviews[0]

(1 point) Train and evaluate your model again with these pretreatment.

In [ ]:
pipeline.fit(X_train_no_stop, y_train)
pipeline.score(X_train_no_stop, y_train)

In [ ]:
X_test_no_stop = remove_stop_words(X_test)
pipeline.score(X_test, y_test)

Here we have a strange behavior where the presence of stopwords in our testing data causes to lose a large amount of accuracy

In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

In [ ]:
def stem_reviews(reviews):
  stemmed_l = []
  for review in reviews:
    stemmed = [stemmer.stem(elt) for elt in review.split()]
    stemmed = ' '.join(stemmed)
    stemmed_l.append(stemmed)
  return stemmed_l

X_train_stem = stem_reviews(X_train)

In [ ]:
X_train_stem[0]

In [ ]:
X_test_stem = stem_reviews(X_test)
pipeline.fit(X_train_stem, y_train)
pipeline.score(X_train_stem, y_train)
pipeline.score(X_test_stem, y_test)

We will now try to apply similar methods but with lemming and compare results.

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

In [ ]:
def lemma_reviews(reviews):
  lemm_l = []
  for review in reviews:
    lem = [lemmatizer.lemmatize(elt) for elt in review.split()]
    lem = ' '.join(lem)
    lemm_l.append(lem)
  return lemm_l

In [ ]:
X_train_lem = lemma_reviews(X_train)
X_test_lem = lemma_reviews(X_test)
pipeline.fit(X_train_lem, y_train)
pipeline.score(X_train_lem, y_train)
pipeline.score(X_test_lem, y_test)

(1 point) Are the results better or worse? Try explaining why the accuracy changed.

In our situation it seems the accuracy barely changed. The main advantage of the lemma and stemming techniques is to reduce the volume of our vocabulary. If the corpora was overly large, we may have a situation where lemming/stemming could have brought a gain in accuracy. However in our situation the fact words are inflected adds better context and therefore aids the model in its precision.